<a href="https://colab.research.google.com/github/CBaffelli/CAS-NLP_Machine-translation/blob/main/03_CAS_NLP_Final_project_multilingual_model_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate sacrebleu accelerate -U bert_score rouge_score peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 39.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset, concatenate_datasets, DatasetDict
import evaluate
import numpy as np
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
#Load the datasets
italian = pd.read_csv('/content/drive/MyDrive/CAS NLP/Final_project/Dataset/data_for_training/italian.csv', dtype=str)
french = pd.read_csv('/content/drive/MyDrive/CAS NLP/Final_project/Dataset/data_for_training/french.csv', dtype=str)
spanish = pd.read_csv('/content/drive/MyDrive/CAS NLP/Final_project/Dataset/data_for_training/spanish.csv', dtype=str)
romanian = pd.read_csv('/content/drive/MyDrive/CAS NLP/Final_project/Dataset/data_for_training/romanian.csv', dtype=str)
portuguese = pd.read_csv('/content/drive/MyDrive/CAS NLP/Final_project/Dataset/data_for_training/portuguese.csv', dtype=str)

In [ ]:
#Create a mapping to iterate in the dataframes
languages = {
    'Italian': italian,
    'French': french,
    'Spanish' : spanish,
    'Romanian' : romanian,
    'Portuguese' : portuguese
}


In [ ]:
#Get the tokenizer and the model
checkpoint = "Helsinki-NLP/opus-mt-en-roa"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/786k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/793k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/295M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
#Creates a mapping for the prefixes
prefix_mapping_T5 = {
    'Italian' : 'translate English to Italian: ',
    'French' : 'translate English to French: ',
    'Spanish' : 'translate English to Spanish: ',
    'Romanian' : 'translate English to Romanian: ',
    'Portuguese' : 'translate English to Portuguese: '
}

prefix_mapping_OPUS =  {
    'Italian' : '>>ita<< ',
    'French' : '>>fra<< ',
    'Spanish' : '>>spa<< ',
    'Romanian' : '>>ron<< ',
    'Portuguese' : '>>por<< '
}

In [ ]:
#Function to transform the dataset and append the correct prefix
##Returns the dataset in the Hugginface dataset format
##This function also adds additional data from the europa_ecdc_tm corpus and the ELRC-Medical-V2 corpus
def transform_dataset_append_prefix_additional(name, df):
  dataset = []
  additional_data = load_dataset("europa_ecdc_tm", language_pair=("en", language_mapping[name]))
  language_pairs = f'en-{language_mapping[name]}'
  additional_data_2 = load_dataset("qanastek/ELRC-Medical-V2", language_pairs)
  prefix = ''
  if 'opus' in checkpoint:
    prefix = prefix_mapping_OPUS[name]
  elif 't5' in checkpoint:
    prefix = prefix_mapping_T5[name]
  for index, row in df.iterrows():
    translation = {'en': prefix + row['sourceExpression'], 'target': row['targetExpression']}
    data = {'translation': translation}
    dataset.append(data)
  for example in additional_data['train']:
    translation = {'en' : prefix + example['translation']['en'], 'target' : example['translation'][language_mapping[name]]}
    data = {'translation': translation}
    dataset.append(data)
  for example in additional_data_2['train']:
    translation = {'en' : prefix + example['translation']['en'], 'target' : example['translation'][language_mapping[name]]}
    data = {'translation': translation}
    dataset.append(data)
  return Dataset.from_pandas(pd.DataFrame(data=dataset))

In [ ]:
#Function to transform the dataset and append the correct prefix
##Returns the dataset in the Hugginface dataset format
def transform_dataset_append_prefix(name, df):
  prefix = ''
  dataset = []
  if 'opus' in checkpoint:
    prefix = prefix_mapping_OPUS[name]
  elif 't5' in checkpoint:
    prefix = prefix_mapping_T5[name]
  for index, row in df.iterrows():
    translation = {'en': prefix + row['sourceExpression'], 'target': row['targetExpression']}
    data = {'translation': translation}
    dataset.append(data)
  return Dataset.from_pandas(pd.DataFrame(data=dataset))

In [ ]:
#Function to preprocess and tokenize the data
##We need to preprocess and prepare the data for the fine-tuning
max_length = 128
source_lang = "en"
def preprocess_function(examples):
  inputs = [example[source_lang] for example in examples["translation"]]
  targets = [example['target'] for example in examples["translation"]]
  model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
  return model_inputs


In [ ]:
#Transform the dataset in the Huggingface dataset format, apply the tokenization
datasets = {}
for language_name, language_df in languages.items():
  initial_dataset = transform_dataset_append_prefix(language_name, language_df)
  datasets[language_name] = initial_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/313067 [00:00<?, ? examples/s]

Map:   0%|          | 0/280419 [00:00<?, ? examples/s]

Map:   0%|          | 0/301191 [00:00<?, ? examples/s]

Map:   0%|          | 0/121782 [00:00<?, ? examples/s]

Map:   0%|          | 0/276939 [00:00<?, ? examples/s]

In [ ]:
#Once we have prepared all the datasets, we can combine them together in a single dataset
combined_dataset = concatenate_datasets(list(datasets.values()))

In [ ]:
combined_dataset

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1293398
})

In [ ]:
combined_dataset['translation']

[{'en': '>>ita<< A "\\" on the module indicates the module is service masked.',
  'target': 'Un segno "\\" sul modulo indica che il modulo è escluso per manutenzione.'},
 {'en': '>>ita<< A "history" function registers which panels have been displayed so far, use the back and forward buttons to display the previous or next panel within this history.',
  'target': 'Una funzione "cronologia" registra quali pannelli sono stati visualizzati fino a quel momento; utilizzare i pulsanti avanti e indietro per visualizzare il pannello precedente o successivo nell\'ambito della cronologia.'},
 {'en': '>>ita<< A - min. 3 mm (barcode quiet zone)',
  'target': 'A - min. 3 mm (zona libera del barcode)'},
 {'en': '>>ita<< A . period or , comma character is included in the search as it is.',
  'target': "Viene incluso nella ricerca, così com'è, un carattere di punto . o virgola ,."},
 {'en': '>>ita<< A 0.01 mM EDTA solution that is contained in a 50 L carboy and is used to fill the liquid system.',
  't

In [ ]:
#Then we split the data into train and test set
#Split 20% for testing
train_test_split = combined_dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
#Create a DatasetDict to hold the splits
final_dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

In [ ]:
final_dataset['train']['translation'][1:10]

[{'en': '>>por<< The process starts with the most recent ones, and once the time is over, the process stops.',
  'target': 'O processo começa com as mais recentes e, assim que o tempo se esgota, o processo é parado.'},
 {'en': '>>fra<< After LDL goal is reached, secondary goal for nonHDL shall be set at 30 mg/dL (0.78 mmol/L) higher than LDL goal.',
  'target': "Une fois l'objectif LDL atteint, le second objectif pour le cholestérol nonHDL doit être fixé à 30 mg/dL (0.78 mmol/L) plus haut que l'objectif LDL."},
 {'en': '>>por<< Displays the linearity degree of 9 or above measurement point.',
  'target': 'Indica o grau de linearidade do ponto de medição 9 ou superior.'},
 {'en': '>>por<< Evaporation of samples or collection media may lead to incorrect or invalid results.',
  'target': 'A evaporação de amostras ou meios de colheita pode levar a resultados incorretos ou inválidos.'},
 {'en': '>>por<< The related functions provide quick access to other information that is associated with t

In [ ]:
#We create a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
#We import the score for the evaluation
sacrebleu_score = evaluate.load("sacrebleu")
rouge_score = evaluate.load("rouge")
meteor_score = evaluate.load('meteor')
bertscore = evaluate.load("bertscore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
#Function to post-process text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

#Function to compute the metrics
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = rouge_score.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        rouge_types=["rouge1", "rouge2", "rougeL"]
    )
    score = sacrebleu_score.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )

    meteor_result = meteor_score.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )

    bert_scores = bertscore.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        model_type="bert-base-multilingual-cased")

    result["sacrebleu"] = score["score"]
    result['meteor'] = meteor_result["meteor"]
    result['bert precision'] = np.mean(bert_scores['precision'])
    result['bert recall'] = np.mean(bert_scores['recall'])
    result['bert f1'] = np.mean(bert_scores['f1'])

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
#Freeze all layers except the task-specific layers (decoder layers in this case)
for name, param in model.named_parameters():
    if 'shared' in name:  # 'shared' is commonly used for the embedding parameter
        param.requires_grad = False

In [ ]:
#Load the model with the PEFT config
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules = ['k_proj', 'v_proj', 'q_proj', 'out_proj']
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 589,824 || all params: 74,268,160 || trainable%: 0.7941815173554858


In [ ]:
#Hyperparameters and misc
learning_rate = 1e-03
batch_size = 16
epochs = 4

In [ ]:
#We define the training arguments and the trainer
training_args = Seq2SeqTrainingArguments(
    output_dir='OPUS_batch16',
    evaluation_strategy="no",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    gradient_accumulation_steps=4,
    dataloader_num_workers=4,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss
500,1.727400
1000,1.683700
1500,1.626200
2000,1.585500
2500,1.517000
3000,1.515700
3500,1.460600
4000,1.432800
4500,1.415100
5000,1.361500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[56670]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[56670]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

Step,Training Loss
500,1.727400
1000,1.683700
1500,1.626200
2000,1.585500
2500,1.517000
3000,1.515700
3500,1.460600
4000,1.432800
4500,1.415100
5000,1.361500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[56670]], 'forced_eos_token_id': 0}


TrainOutput(global_step=64668, training_loss=0.8563037758347807, metrics={'train_runtime': 16323.8433, 'train_samples_per_second': 253.548, 'train_steps_per_second': 3.962, 'total_flos': 6.002546772580762e+16, 'train_loss': 0.8563037758347807, 'epoch': 3.9998762950363385})

In [ ]:
trainer.save_model('/content/drive/MyDrive/CAS NLP/Final_project/Model/OPUS_batches16')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[56670]], 'forced_eos_token_id': 0}
